# 结构型-适配器模式

## 模式说明

适配器模式主要用来提供兼容性的接口，它和外观模式有点像，区别在于，适配器模式对一个现有的接口匹配到另一个，而外观模式试图从一个复杂的系统中抽象出一个简单的接口。

## 实例

### 实例1

In [27]:
# 外部类1,无法修改，只能扩展
class Dog:
    def __init__(self, name):
        self.name = name

    def bark(self):
        print(f"{self.name}: 汪汪汪！")


# 外部类2
class Duck:
    def __init__(self, name):
        self.name = name

    def chirp(self):
        print(f"{self.name}: 嘎嘎嘎！")


# 现在在我们自己的程序中，提供统一的speak方法
class Adapter:
    def __init__(self, obj, methods_mapping):
        self.obj = obj
        self.__dict__.update(methods_mapping)


dog = Dog("富贵")
duck = Duck("川普")

objects = [
    Adapter(dog, {"speak": dog.bark}),
    Adapter(duck, {'speak': duck.chirp})
]
for obj in objects:
    obj.speak()

富贵: 汪汪汪！
川普: 嘎嘎嘎！


基本的思路就是做一个适配器和原方法之间的映射。

这里有一个细节容易让人困扰，通过`adapter.speak`调用的时候，传递给方法的`self`不应该是`adapter`实例吗？为什么代码可以正确的执行？这是因为`dog.bark`以及`duck.chirp`返回的是一个绑定的方法，也就是说，`bark`方法已经和`dog`这个实例绑定了，比如：

In [26]:
bark = dog.bark
bark()

富贵: 汪汪汪！


当我们调用`adapter.speak`时，返回的是bound method。如果像下面这样写，则会报错：

In [31]:
adapter = Adapter(dog, {'speak': Dog.bark})
try:
    adapter.speak()
except TypeError as e:
    print(e)

bark() missing 1 required positional argument: 'self'


想象中应该出现的是没有`name`属性，结果提示缺少`self`参数，仔细想想，原因是： 
更新`__dict__`，其实是添加到`adapter`实例的命名空间，而函数必须是类的属性的时候，才会返回一个绑定方法。所以`adapter.speak`只是简单的在`adapter`的命名空间中查找`speak`对应的值，并不会返回一个绑定方法，而`Dog.bark`也仅仅就是一个函数。所以`adapter.speak()`不会传递`self`。

这和javascript的this动态绑定有很大区别，所以容易造成困惑。

### 实例2

设计模式是一种思路，代码编写并没有固定的模式，适配器模式的精髓是将一个接口匹配到另一个，比如下面的实例，也是适配器模式：

In [1]:
class AgeCalculator:
    def __init__(self, birthday):
        self.year, self.month, self.day = (int(x) for x in birthday.split('-'))
        
    def calculate_age(self, date):
        year, month, day = (int(x) for x in date.split('-'))
        age = year - self.year
        if (month, day) < (self.month, self.day):
            age -= 1
        return age

上面的代码实现了年龄之间的计算，但是它只接受字符串形式的日期，但是在python中，我们更多的使用`datetime`对象，可是如果向`calculate_age`方法传递一个`datetime`对象，显然会报错。因此，我们可以提供一个适配器，实现`datetime`对象的年龄计算。

In [12]:
from datetime import datetime

class DateAgeCaculator:
    @staticmethod
    def _str_date(date):
        return date.strftime("%Y-%m-%d")
        
    def __init__(self, birthday):
        birthday = self._str_date(birthday)
        self.caculator = AgeCalculator(birthday)
        
    def calculate_age(self, date):
        date = self._str_date(date)
        return self.caculator.calculate_age(date)

我们创建了一个`DateAgeCaculator`的类，对`AgeCalculator`进行了封装，在内部实现了`datetime`类型和字符串之间的转换：

In [14]:
birthday = datetime(year=1979, month=10, day=2)
now = datetime.now()

caculator = DateAgeCaculator(birthday)
caculator.calculate_age(now)

41

## 总结

以上两种方法都是适配器模式，都是创建一个新类对原有类进行封装，将一个方法匹配到另一个。区别在于：
- 第一种方法是显示的创建原有类的实例，其方法之间的映射关系也是通过参数传递给新类，新类只是简单的在方法名称之间进行了一个映射。
- 第二种方法在新类的内部创建原有类实例，而且新类不仅只是方法名进行了映射，还进行了一些额外的操作。